In [1]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-557077ab-3e77-461f-abd8-8868ce34cf39;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (133ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (33ms)
:: resolution report :: resolve 732ms :: artifacts dl 169ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	------

23/03/22 01:32:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/22 01:32:09 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [7]:
%run trace_encode.ipynb

In [2]:
#first question I have is how many events do I have when I call readfirst
df = readFirstEvents()

 1:38AM UTC on Mar 22, 2023 --- read time: 30.113750457763672 seconds ---


In [4]:
#what is our count
df.count()

948334

In [3]:
#how many days do we have
df.select('event_day').distinct().show()

+---------+
|event_day|
+---------+
|       23|
+---------+



In [5]:
#how many hours
df.select('event_hour').distinct().show()

+----------+
|event_hour|
+----------+
|        13|
|        16|
|        19|
|        15|
|        17|
|        14|
|        18|
+----------+



In [8]:
#how many malicious events do you have? 
df_m = df.where(col('malicious') == 1)
df_m.count()

3624

In [9]:
#does first events work now on data duplication
cg = create_graph(df_m)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 1261


In [10]:
cg.limit(10).toPandas()
#answer I belive it did. 

,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, 36d062d...","(36d062d4-1f69-4dcc-9791-46b5c07dadef,)"
1,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4b9755a...","(4b9755ad-05bb-458e-b9ff-27852eb51860,)"
2,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 50ac366...","(50ac366a-4500-4ef0-8579-22adacaf6438,)"
3,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 6fba2e2...","(6fba2e2c-1552-41ad-80ee-fffdfbe58d29,)"
4,"(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 4aec590...","(4aec5900-4572-4d09-81e6-fbc227a7f58d,)","(4aec5900-4572-4d09-81e6-fbc227a7f58d, 3eea907...","(3eea9071-9ceb-468a-ad89-9e3890833e52,)","(3eea9071-9ceb-468a-ad89-9e3890833e52, 75c93ca...","(75c93ca7-955f-4e24-b728-cb5dd9ed1d2e,)"
5,"(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, 7f91e81...","(7f91e814-3f77-4ee9-bfee-01d796b8f424,)"
6,"(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245cc7,)","(a52f6519-6265-4437-9f6c-3402fb245cc7, f5d1c00...","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2,)","(f5d1c003-1f65-4a8a-ba8c-413da99d70f2, a52f651...","(a52f6519-6265-4437-9f6c-3402fb245